In [9]:
%pip install neat-python

  Using cached neat_python-0.92-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas
import random
import numpy as np
import itertools
import os
import neat

with open('valid_guesses.csv') as f:
    guesses = f.read().splitlines()
with open('valid_solutions.csv') as f:
    solutions = f.read().splitlines()


print(len(solutions))

2315


In [11]:
class LetterPos:
    def __init__(self, letter, pos, gpos, perfect):
        self.letter = letter
        self.pos = pos
        self.gpos = gpos
        self.perfect = perfect

class GameState:
    def __init__(self):
        self.guesses = []
        self.solution = random.choice(solutions)
        self.correct_letters = []
        self.incorrect_letters = []
        self.turn = 0
        self.state = True

    def output_turn(self):
        if self.correct_letters[self.turn] is not None:
            for i in range(len(self.correct_letters[self.turn])):
                print(self.correct_letters[self.turn][i].letter)
                print(self.correct_letters[self.turn][i].pos)
                print(self.correct_letters[self.turn][i].perfect)

    def pretty_output(self):
        guess_result = list("⬛⬛⬛⬛⬛")
        if self.correct_letters[self.turn] is not None:
            for i in range(len(self.correct_letters[self.turn])):
                guess_result[self.correct_letters[self.turn][i].gpos] = "🟩" if self.correct_letters[self.turn][i].perfect else "🟨"
        print(''.join(guess_result))

    def globalHints(self):
        return list(itertools.chain(*self.correct_letters))

    def hintData(self):
        correct_letters = self.globalHints()
        hint_data = []

        for i in range(len(correct_letters)):
            hint_data.append([correct_letters[i].letter, correct_letters[i].pos, correct_letters[i].gpos, correct_letters[i].perfect])

        return hint_data

    def perfectLetters(self):
        perfectLetters = []
        for i in range (self.turn):
            for j in range(len(self.correct_letters[i])):
                if self.correct_letters[i][j].perfect == True:
                    perfectLetters.append(self.correct_letters[i][j])
                    
        return perfectLetters

    def nextTurn(self):
        if self.turn < 5:
            self.turn += 1
        else:
            self.turn += 1
            self.state = False
    
    def endgame(self):
        print("You won in : " +  str(self.turn + 1))
        self.state = False
     

In [12]:
def compare(game, guess, solution):
    #compare the two words
    print(guess)
    guess_letters = list(guess)
    solution_letters = list(solution)
    
    if(guess_letters == solution_letters):
        print("🟩🟩🟩🟩🟩")
        game.endgame()
        return
    
    correct_letters = []
    incorrect_letters = []
    #check for letter matches
    for i in range(5):
        for j in range(5):
            if(solution_letters[i] == guess_letters[j]):
                guess_grade = LetterPos(guess_letters[j], i, j, (i == j))
                correct_letters.append(guess_grade)
                break

    #check for letter misses
    for i in range(5):
        if(guess_letters[i] not in solution_letters):
            incorrect_letters.append(guess_letters[i])
            
    game.incorrect_letters = list(set(game.incorrect_letters + incorrect_letters))

    #edge cases
    #check for repeating LetterPos
    #if two LetterPos classes have the same letter and pos pop one
    # for i in range(len(correct_letters)):
    #     for j in range(len(correct_letters)):
    #         if(correct_letters[i].pos == correct_letters[j].pos and i != j):
    #             correct_letters.pop(j)

    #win condition
    #check if the guess is perfect
    # if(len(correct_letters) == 5):
    #     isPerfect = True
    #     for i in range(len(correct_letters)):
    #         if(correct_letters[i].perfect == False):
    #             isPerfect = False
    #     if(isPerfect):


    
    game.correct_letters.append(correct_letters)

    

In [13]:
#game loop
def wordlegame():
    game = GameState()
    #randomly select a word from the list solutions
    print(game.solution)
    while(game.state):
        # guess = random_guess()
        # guess = elimination_guess(game)
        # guess = kinda_smart(game)
        guess = smart_AI(game)

        #compare the guess to the solution
        compare(game, guess, game.solution)
        if(game.state):
            game.pretty_output()
            #print(game.incorrect_letters)
            game.nextTurn()
    
    return game.turn + 1


In [5]:
#stat calculations
running_sum = 0
wins = 0
losses = 0

for i in range (1000):
    score = wordlegame()
    running_sum += score
    if score == 6:
        wins += 1
    else:
        losses += 1

score_avg = running_sum / 1000

print("Average Score: " + str(score_avg))
print("Winrate: " + str((wins/1000) * 100)+ "%")


pause


NameError: name 'smart_AI' is not defined

In [14]:
def random_guess():
    guess = random.choice(guesses)
    return guess

In [15]:
def elimination_guess(game):
    possible_words = []


    possible_words = [ele for ele in solutions if all(ch not in ele for ch in game.incorrect_letters)]

    guess = random.choice(possible_words)
    
    return guess

In [16]:
def kinda_smart(game):
    #for the first 3 turns, guess words without incorrect letters
    if(game.turn < 3):
        guess = elimination_guess(game)
        return guess
    #for the next 3 guess words with the correct letters and without incorrect letters
    else:
    #process the correct letter objects, words must contain those letters (no positions) and no incorrect letters
        correct_chars = []
        correct_letters = game.globalHints()
        for i in range(len(correct_letters)):
            correct_chars.append(correct_letters[i].letter)
        #print(correct_chars)

        possible_words = [ele for ele in solutions if all(ch not in ele for ch in game.incorrect_letters)]
        possible_solutions = [ele for ele in possible_words if all(ch in ele for ch in correct_chars)]

        guess = random.choice(possible_solutions)
        
        return guess


    

In [17]:
def smart_AI(game):
    #for the first 3 turns, guess words without incorrect letters
    if(game.turn < 3):
        guess = elimination_guess(game)
        return guess
    #for the next 3 guess words with the correct letters and without incorrect letters
    else:
    #process the correct letter objects, words must contain those letters (no positions) and no incorrect letters
        correct_chars = []
        correct_letters = game.globalHints()

        hint_data = []

        for i in range(len(correct_letters)):
            hint_data.append([correct_letters[i].letter, correct_letters[i].pos, correct_letters[i].gpos, correct_letters[i].perfect])

        for i in range(len(correct_letters)):
            correct_chars.append(correct_letters[i].letter)
        #print(correct_chars)
        print(hint_data)



        possible_words = [ele for ele in solutions if all(ch not in ele for ch in game.incorrect_letters)]
        possible_solutions = [ele for ele in possible_words if all(ch in ele for ch in correct_chars)]

        perfectLetters = game.perfectLetters()

        #output perfect letters
        # for i in range(len(perfectLetters)):
        #     print(perfectLetters[i].letter + str(perfectLetters[i].pos))

        better_solutions = []

        if (len(perfectLetters) == 0):
            better_solutions = possible_solutions
        else:
            for i in range(len(possible_solutions)):
                for j in range(len(perfectLetters)):
                    if(possible_solutions[i][perfectLetters[j].pos] == perfectLetters[j].letter):
                        better_solutions.append(possible_solutions[i])


        # print(possible_solutions)
        # print(better_solutions)
        
        
        guess = random.choice(better_solutions)
        
        return guess



In [26]:
wordlegame()

vowel
dingo
⬛⬛⬛⬛🟨
hotel
⬛🟩⬛🟩🟩
locus
🟨🟩⬛⬛⬛
[['o', 1, 4, False], ['o', 1, 1, True], ['e', 3, 3, True], ['l', 4, 4, True], ['o', 1, 1, True], ['l', 4, 0, False]]
lower
🟨🟩🟩🟩⬛
[['o', 1, 4, False], ['o', 1, 1, True], ['e', 3, 3, True], ['l', 4, 4, True], ['o', 1, 1, True], ['l', 4, 0, False], ['o', 1, 1, True], ['w', 2, 2, True], ['e', 3, 3, True], ['l', 4, 0, False]]
vowel
🟩🟩🟩🟩🟩
You won in : 5


5

In [18]:
def run(config_path):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                            neat.DefaultSpeciesSet, neat.DefaultStagnation,
                            config_path)

    p = neat.Population(config)

    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    winner = p.run(wordlegame_evo,50)

    print(winner)


    local_dir = os.path.dirname(__file__)
    config_path = os.path.join(local_dir, "config-feedforward.txt")
    run(config_path)

In [23]:
#game loop
def wordlegame_evo(genomes, config):
    #game = GameState()

    nets = []
    ge = []
    games = []

    for _,g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        games.append(GameState())
        g.fitness = 0
        ge.append(g)
        

    #randomly select a word from the list solutions
    for game in games:
        while(game.state):
            # guess = random_guess()
            # guess = elimination_guess(game)
            # guess = kinda_smart(game)
            # guess = smart_AI(game)

            #inputs
            # 0: correct_letters
            # 1: incorrect_letters
            # 2: current_turn

            inputs = []

            hint_data = game.hintData()
            for i in range(len(hint_data)):
                #letter to numerical valu 0-25
                if(hint_data[i].perfect == True):
                    inputs.append(ord(hint_data[i].letter) - 97)
                else:
                    inputs.append((ord(hint_data[i].letter) - 97 )*2) 
            #for each letter

            if(len(inputs) < 5):
                #append to the end of the list
                for i in range(5 - len(inputs)):
                    inputs.append(0)
            
            for i in range(len(game.incorrect_letters)):
                inputs.append(ord(game.incorrect_letters[i]) - 97)
            
            if(len(inputs) < 31):
                #append to the end of the list
                for i in range(30 - len(inputs)):
                    inputs.append(0)
            
            inputs.append(game.turn)

            guess = net.activate(inputs)


            #compare the guess to the solution
            compare(game, guess, game.solution)
            if(game.state):
                game.pretty_output()
                #print(game.incorrect_letters)
                game.nextTurn()
        
            return game.turn + 1
        
        g.fitness += 7 - game.turn




In [ ]:
run("config-feedforward.txt")